# Masking a column

El objetivo de este notebook es mostrar la creacion y aplicacion de una funcion 
para enmascarar una columna en un dataset, con la particularidad de que ante el mismo dato se produzca siempre la misma mascara, o en este caso el mismo hash, permitiendo asi la relacion de observaciones entre diferentes datasets.

Como funcion particular para este Hash se eligio [MD5](https://en.wikipedia.org/wiki/MD5). Este algoritmo es generalmente usado para verificar la integridad de un dato, nunca debe ser usado para encriptar passwords ni nada por el estilo. 

En este caso lo elegimos porque es un algoritmo que require poco recursos para ejecutarse, tiene una extension de 128 bits, este significa que si tenemos un dataset con N cantidad de observaciones, recien a la numero: 18446744073709551616 (2^64) tendremos una [probabilidad de 50% de colision](https://auth0.com/blog/birthday-attacks-collisions-and-password-strength/) 

Sin embargo la aplicacion de otras funciones hash como SHA128 pueden implentarse de igual forma a la demostrada aca. 


Como una primera demostracion vemos como funcian de forma aislada el algoritmo. 

Algo a tener encuenta es que la funcion de hash requiere bytes en vez de string (por eso la "b" antes del string). Veremos unas celdas mas abajo como hacer un encoding de string a bytes. 

In [4]:
import hashlib

result = hashlib.md5(b"prueba")
result.hexdigest()

'c893bad68927b457dbed39460e6afd62'

Generamos un dataframe de prueba que solo tiene dos columnas: dni y age

In [6]:
import pandas as pd
data = {
    "dni": ["44.555.555", "33.444.555", "21.333.444"], "age": [5, 30, 20]
}
df = pd.DataFrame(data=data)
df.head()

,dni,age
0,44.555.555,5
1,33.444.555,30
2,21.333.444,20


Creamos la funcion que aplicaremos luego al dataset usando la funcion apply() de pandas

Ademas agregamos la verificacion de que la aplicacion de la funcion de hash al mismo dato genera el mismo hash.

In [14]:
def hash_mask(data):
    """ 
    Funcion que genera un HASH MD5 de un dato recibido
    Como solo se puede aplicar encoding a un string primero convertimos
    el dato recibido a string. 
    """
    data_str = str(data)
    hashed = hashlib.md5(data.encode("utf-8"))
    return hashed.hexdigest()

one_hash = hash_mask(data["dni"][0])
other_hash = hash_mask(data["dni"][0])
assert one_hash == other_hash
one_hash

'28f17d4498afc74e8665cb7e98c421aa'

Finalmente lo aplicamos al dataset:

In [8]:
df["dni"].apply(dni_mask)

0    28f17d4498afc74e8665cb7e98c421aa
1    214d3ebe560722b9ae7f906bcc5c046a
2    c605d20c10d4de0c3829d498366b6a2f
Name: dni, dtype: object

In [9]:
df["dni_mask"] = df["dni"].apply(dni_mask)

In [10]:
df.head()

,dni,age,dni_mask
0,44.555.555,5,28f17d4498afc74e8665cb7e98c421aa
1,33.444.555,30,214d3ebe560722b9ae7f906bcc5c046a
2,21.333.444,20,c605d20c10d4de0c3829d498366b6a2f
